In [1]:
import os, sys
import pandas as pd
import sqlite3

In [2]:
tsv, ref = "./cache/scratch_hits.eg.tsv", "../../../lx_ref/eggnog.db"

columns = f"qseqid stitle qstart qend nident pident evalue".split(" ")

df_an = pd.read_csv(tsv, sep="\t", header=None)
df_an.columns = columns

df_an

,qseqid,stitle,qstart,qend,nident,pident,evalue
0,k59_264_1,292564.Cyagr_2032,2,72,43,60.6,1.100000e-23
1,k59_264_1,1407650.BAUB01000028_gene2817,1,72,42,58.3,6.370000e-21
2,k59_264_1,197221.22295135,1,72,41,56.9,8.330000e-19
3,k59_264_1,32057.KB217478_gene5889,4,72,37,52.1,5.870000e-16
4,k59_264_1,1170562.Cal6303_3076,1,72,39,52.7,1.120000e-15
...,...,...,...,...,...,...,...
81401,k59_219_12,1348662.CARG_02845,14,45,27,84.4,1.060000e-08
81402,k59_219_12,1306990.BARG01000013_gene1015,4,50,28,59.6,1.090000e-08
81403,k59_219_12,1449347.JQLN01000005_gene3963,14,46,27,81.8,1.180000e-08
81404,k59_219_12,680198.SCAB_53051,5,46,27,64.3,1.680000e-08


In [5]:
with sqlite3.connect(ref) as con:
    cur = con.cursor()
    _max, _max_og = 0, None
    for row in cur.execute(f"select * from og"):
        og, level, nm, desc, cog = row
        nm = int(nm)
        if nm > _max:
            _max = nm
            _max_og = og
        
    res = next(cur.execute(f"select * from og where og='{_max_og}'"))
    print(res)
# seems like nm is size of cluster

('COG0477', '1', 125439, 'Major facilitator Superfamily', 'EGP')


In [42]:
prot_cols = ['name', 'bigg_reaction', 'gos', 'pfam', 'pname', 'ogs', 'orthoindex', 'kegg_ko', 'kegg_cog', 'kegg_disease', 'kegg_ec', 'kegg_brite', 'kegg_rclass', 'kegg_tc', 'kegg_cazy', 'kegg_pathway', 'kegg_module', 'kegg_reaction', 'kegg_go', 'kegg_drug', 'kegg_pubmed', 'kegg_network']
group_cols = ['og', 'level', 'nm', 'description', 'COG_categories']

with sqlite3.connect(ref) as con:
    cur = con.cursor()

    def _get_prot(k):
        return next(cur.execute(f"select * from prots where name='{k}'"))
    
    def _get_grp(k, l):
        return next(cur.execute(f"select * from og where og='{k}' and level='{l}'"))
    
    gi = prot_cols.index('ogs')
    prots = []
    group_ks = set()
    for _, row in df_an.iterrows():
        key = row.stitle
        annot = _get_prot(key)
        _groups = annot[gi] if annot[gi] is not None else ""
        group_ks.update([tuple(g.split("@")) for g in _groups.split(",") if "COG" not in g])
        prots.append(annot)
    dfp = pd.DataFrame(prots, columns=prot_cols)
    dfp.to_csv("./cache/test.annots.csv", index=False)

    groups = []
    for gk, level in group_ks:
        groups.append(_get_grp(gk, level))
    dfg = pd.DataFrame(groups, columns=group_cols)
    dfg.to_csv("./cache/test.groups.csv", index=False)

In [13]:
def do(q):
    with sqlite3.connect(ref) as con:
        cur = con.cursor()

        x = cur.execute(q)
        return x
    
for i, x in enumerate(do(f"select * from og")):
    print(x)
    if i > 3: break
print()
for i, x in enumerate(do(f"select * from prots")):
    print(x)
    if i > 3: break

('1EQ0F', '1016', 12, 'Methyltransferase domain protein', 'Q')
('1EQ0G', '1016', 5, 'Psort location CytoplasmicMembrane, score', 'V')
('1EQ0H', '1016', 12, 'This is 1 of the proteins that binds and probably mediates the attachment of the 5S RNA into the large ribosomal subunit, where it forms part of the central protuberance. In the 70S ribosome it contacts protein S13 of the 30S subunit (bridge B1b), connecting the 2 subunits', 'J')
('1EQ0I', '1016', 11, 'Belongs to the FBPase class 1 family', 'G')
('1EQ0J', '1016', 10, 'Transporter, DASS family', 'P')

('#member', None, None, None, None, None, None, 'KOs', 'cog', 'disease', 'enzyme', 'brite', 'rclass', 'tc', 'cazy', 'pathway', 'module', 'reaction', 'go', 'drug', 'pubmed', 'network')
('1000565.METUNv1_00006', None, None, 'DDE_Tnp_1,DUF772', 'EGK73635.1', '1MUVI@1224,2KUE1@206389,2VK9J@28216,COG3039@1,COG3039@2', '3136246,3136328,3136329,3136330,3136331,3136341,3136342,20677502,20677503,20677504,29171134,29171139,29171140,29171146,4044

In [15]:
for i, x in enumerate(do(f"select * from sqlite_schema")):
    print(x)

('table', 'event', 'event', 2, 'CREATE TABLE event(\ni INTEGER PRIMARY KEY,\nlevel VARCHAR(16),\nog VARCHAR(16),\nside1 TEXT,\nside2 TEXT)')
('table', 'og', 'og', 3, 'CREATE TABLE "og"(og VARCHAR(16), level VARCHAR(16), nm INTEGER, description TEXT, COG_categories VARCHAR(8), PRIMARY KEY (og, level)) without rowid')
('table', 'version', 'version', 4, 'CREATE TABLE "version" (version VARCHAR(16) PRIMARY KEY) without rowid')
('table', 'prots', 'prots', 5, 'CREATE TABLE "prots" (name VARCHAR(32) PRIMARY KEY, bigg_reaction VARCHAR(32), gos TEXT, pfam TEXT, pname VARCHAR(32), ogs VARCHAR(256), orthoindex VARCHAR(256), kegg_ko VARCHAR(256), kegg_cog VARCHAR(256), kegg_disease VARCHAR(256), kegg_ec VARCHAR(256), kegg_brite VARCHAR(256), kegg_rclass VARCHAR(256), kegg_tc VARCHAR(256), kegg_cazy VARCHAR(256), kegg_pathway VARCHAR(256), kegg_module VARCHAR(256), kegg_reaction VARCHAR(256), kegg_go VARCHAR(256), kegg_drug VARCHAR(256), kegg_pubmed TEXT, kegg_network TEXT) WITHOUT ROWID')
('index'

In [30]:
_prots = "name VARCHAR(32) PRIMARY KEY, bigg_reaction VARCHAR(32), gos TEXT, pfam TEXT, pname VARCHAR(32), ogs VARCHAR(256), orthoindex VARCHAR(256), kegg_ko VARCHAR(256), kegg_cog VARCHAR(256), kegg_disease VARCHAR(256), kegg_ec VARCHAR(256), kegg_brite VARCHAR(256), kegg_rclass VARCHAR(256), kegg_tc VARCHAR(256), kegg_cazy VARCHAR(256), kegg_pathway VARCHAR(256), kegg_module VARCHAR(256), kegg_reaction VARCHAR(256), kegg_go VARCHAR(256), kegg_drug VARCHAR(256), kegg_pubmed TEXT, kegg_network TEXT"
group_cols = [v.split(" ")[0] for v in _prots.split(", ")]
print(group_cols)

['name', 'bigg_reaction', 'gos', 'pfam', 'pname', 'ogs', 'orthoindex', 'kegg_ko', 'kegg_cog', 'kegg_disease', 'kegg_ec', 'kegg_brite', 'kegg_rclass', 'kegg_tc', 'kegg_cazy', 'kegg_pathway', 'kegg_module', 'kegg_reaction', 'kegg_go', 'kegg_drug', 'kegg_pubmed', 'kegg_network']


In [32]:
_og = "og VARCHAR(16), level VARCHAR(16), nm INTEGER, description TEXT, COG_categories VARCHAR(8)"
og_cols = [v.split(" ")[0] for v in _og.split(", ")]
print(og_cols)

['og', 'level', 'nm', 'description', 'COG_categories']


In [27]:
with sqlite3.connect(ref) as con:
    cur = con.cursor()
    x = cur.execute(f"select * from og where og='1EQ0I'")
    for i, y in enumerate(x):
        print(y)
        if i > 3: break

with sqlite3.connect(ref) as con:
    cur = con.cursor()
    x = cur.execute(f"select * from prots where name='197221.22295135'")
    for i, y in enumerate(x):
        print(y)
        if i > 3: break

('1EQ0I', '1016', 11, 'Belongs to the FBPase class 1 family', 'G')
('197221.22295135', None, None, 'CAP', 'tll1411', '1G9UW@1117,COG2340@1,COG2340@2', '568081,568082,568083,39675803,39676343,39676344,39676352,39676361,39676362,39676363,39676364,39676369', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '')
